# Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
#代码段中的变量完全显示
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data Exploration Analysis

## Load train and test data sets

In [ ]:
train = pd.read.csv("filepath.csv", encoding="gb18030")
#test = pd.read.csv("filepath.csv", encoding="gb18030")

## Look at few top rows

In [ ]:
train.head(10)

In [ ]:
test.head(5)  #test has no Y column

## Check variable's type

In [ ]:
train.dtypes

## Check dimensions

In [ ]:
train.shape

In [ ]:
test.shape

## Check missing values (write down all the problems)

In [ ]:
train.isnull().sum()

## Variables' distribution (write down all the problems)

### check the missing values of the features
### check the min, mean , max and range of the feature, and try to checkout 
### whether it is has outliers(valuea are in a reasonable range, the meaning of the zero points, the NaN values)

In [ ]:
train.describe(include=[np.number])

## Check all the  columns of object type (write down all the problems)

### Check the feature names, and check whether it has duplicated values or miss match problem

In [ ]:
train['col1'].value_counts()

In [ ]:
train['col2'].value_counts()

# Data Visualization

## Item Visibility vs. Sale (Scatter Graph)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.scatter(x=train['Item_Visibility'], y=train['Item_Outlet_Sales'])
plt.xlabel('Item_Visibility')
plt.ylabel('Item_Outlet_Sales')
plt.show

## Item Type vs. Sales (Bar Chart)

In [ ]:
train[['Item_Type','Item_Outlet_Sales']].groupby('Item_Type').sum().plot.bar(figsize(10,8))

## Item Type vs. Item MRP (Box Plot)

In [ ]:
train[['Item_Type', 'Item_MRP']].boxplot(column='Item_MRP', by='Item_Type', figsize=(10, 8))

# Data Preprocession

## Combine Train and Test Datasets

In [ ]:
full_data = pd.concat([train,test])

In [ ]:
full_data.head()

In [ ]:
train.shape[0]

In [ ]:
test.shape[0]

In [ ]:
full_data.shape[0]

## Impute missing values in features

In [ ]:
full_data['col1'].fillna(full_data['col1'].median(). inplace =True) #fill NaN with median

In [ ]:
full_data['col1'].fillna(full_data['col1'].mean(). inplace =True) #fill NaN with mean

### Check the features whether it has been filled by value

In [ ]:
full_data['col1'].isnull().sum()

## Inputation 0 in feature

In [ ]:
full_data['col2'].replace(0, full_data['col2'].median(), inplace = True)

In [ ]:
full_data['col2'].replace(0, full_data['col2'].mean(), inplace = True)

In [ ]:
(full_data['col2'] ==0).sum()

## Deal with the feature mis-matched categories issue

In [ ]:
full_data['Item_Fat_Content'].value_counts()

In [ ]:
mismatch_dict = {'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat', 'Low Fat':'Low Fat', 'Regular':'Regular'}

In [ ]:
full_data['Item_Fat_Content'].replace(mismatch_dict, inplace=True)

In [ ]:
full_data['Item_Fat_Content'].value_counts() # Check whether it has been fixed

## Rename level in Outlet_Size

In [ ]:
full_data['Outlet_Size'].fillna('other', inplace=True)

In [ ]:
full_data['Outlet_Size'].value_counts()

## Create a new factor: outlet establishment time range

In [ ]:
full_data['Year'] = full_data['Outlet_Establishment_Year'].apply(lambda x:2017-x)

In [ ]:
full_data.head(5)

## Convert categorical variables to dummies

In [ ]:
dummy = pd.get_dummies(full_data[['col1','col2','col3','col4','col5']])

In [ ]:
full_data_ready = pd.concat((full_data, dummy), axis=1)
full_data_ready

# Predictive Modeling

## Drop input variables not required in modeling

In [ ]:
full_vars = list(full_data_ready.columns)
full_vars

## Create target variable

In [ ]:
target_var = 'col0'

## Divide data set

In [ ]:
train_size = train.shape[0]
train_size

In [ ]:
test.shape[0]

In [ ]:
train_x = full_data_ready[full_vars][:train_size].values
train_y = full_data_ready[target_var][:train_size].values
train_x = full_data_ready[full_vars][train_sizes:].values

## Run linear model on train data set

In [ ]:
from sklearn import linear_model
from sklearn import metrics

In [ ]:
lr1 = linear_model.LinearRegression()
lr1.fit(train_x, train_y)

## Calculate Mean squared error

In [ ]:
train_predict = lr1.predict(train_x)

In [ ]:
metrics.mean.absolute_error(train_y, train_predict)

## Calculate R square

In [ ]:
lr1.score(train_x, train_y)

## Residual plot

In [ ]:
plt.scatter(train_predict, train_y - train_predict)

## Deal with heteroskedasticity (解决异方差)

In [ ]:
lr2 = linear_model.LinearRegression()
lr2.fit(train_x, np.log(train_y))

## Calculate Mean squared error

In [ ]:
metrics.mean.absolute_error(train_y, np.exp(lr2.predict(train_x)))

## Calculate R square

In [ ]:
lr2.score(train_x, np.log(train_y))

## Residual plot

In [ ]:
plt.scatter(np.exp.(lr2.predict(train_x)), np.exp(lr2.predict(train_x))-train_y)

## Predict sales for test dataset

In [ ]:
predict_sale = lr2.predict(test_x)

In [ ]:
predict_sale

In [ ]:
df_predict_sale = pd.DataFrame(predict_sale)
df_predict_sale.columns =['pred_sale']
df_predict_sale.head(5)

In [ ]:
pd.concat([test, df_predict_sale], axis=1).head(5)